[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F12%2F12.ipynb)

# Day 12

Firstly, we need some standard packages to read the text file.

In [1]:
with Ada.Text_IO;
with Ada.Integer_Text_IO;

Let's move two ships. The first one will follow rules from Part I, but the second one - rules from Part II. Both of them will have a common control interface, but different implementation. Firstly let's define the interface as an Ada `interface` type. I hope it's self-explanatory.

In [2]:
package Controls is
   type Control_Interface is limited interface;
   procedure Move
     (Self  : in out Control_Interface;
      North : Natural := 0;
      South : Natural := 0;
      East  : Natural := 0;
      West  : Natural := 0) is abstract;

   procedure Turn
     (Self  : in out Control_Interface;
      Left  : Natural := 0;
      Right : Natural := 0) is abstract;

   procedure Forward
     (Self  : in out Control_Interface;
      Value : Positive) is abstract;

   function Manhattan_Distance (Self : Control_Interface) return Natural
     is abstract;
end Controls;

Now declare Part I implementation. The ship contains a position `(East, North)` and an `Angle`. The `Move` will modifies the position, the `Turn` - the `Angle`. The `Forward` modifies the position too, taking the `Angle` into account.

In [3]:
package Part_1 is
   type Ship is new Controls.Control_Interface with private;
private
   type Ship is new Controls.Control_Interface with record
      East  : Integer := 0;
      North : Integer := 0;
      Angle : Integer := 0;
   end record;

   overriding procedure Move
     (Self  : in out Ship;
      North : Natural := 0;
      South : Natural := 0;
      East  : Natural := 0;
      West  : Natural := 0);

   procedure Turn
     (Self  : in out Ship;
      Left  : Natural := 0;
      Right : Natural := 0);

   procedure Forward
     (Self  : in out Ship;
      Value : Positive);

   function Manhattan_Distance (Self : Ship) return Natural;
end Part_1;
      
package body Part_1 is
   overriding procedure Move
     (Self  : in out Ship;
      North : Natural := 0;
      South : Natural := 0;
      East  : Natural := 0;
      West  : Natural := 0) is
   begin
      Self.North := Self.North + North - South;
      Self.East := Self.East + East - West;
   end Move;

   procedure Turn
     (Self  : in out Ship;
      Left  : Natural := 0;
      Right : Natural := 0) is
   begin
      Self.Angle := Self.Angle + Left - Right;
   end Turn;

   procedure Forward
     (Self  : in out Ship;
      Value : Positive) is
   begin
      case Self.Angle mod 360 is
         when 0 =>
            Self.East := Self.East + Value;
         when 90 =>
            Self.North := Self.North + Value;
         when 180 =>
            Self.East := Self.East - Value;
         when 270 =>
            Self.North := Self.North - Value;
         when others =>
            raise Program_Error;
      end case;
   end Forward;

   function Manhattan_Distance (Self : Ship) return Natural is
   begin
      return abs Self.East + abs Self.North;
   end Manhattan_Distance;
end Part_1;

The ship for Part II is a bit different. Besides the ship position, we have also the waypoint position now. Both have the same type, so let's define it as a `Position` record type. We don't need `Angle` any more in the ship state. The `Move` and the `Turn` changes waypoint position, while the `Forward` changes ship position.

In [4]:
package Part_2 is
   type Ship is new Controls.Control_Interface with private;
private
   type Position is record
      East  : Integer;
      North : Integer;
   end record;

   type Ship is new Controls.Control_Interface with record
      Ship     : Position := (0, 0);
      Waypoint : Position := (East => 10, North => 1);
   end record;

   overriding procedure Move
     (Self  : in out Ship;
      North : Natural := 0;
      South : Natural := 0;
      East  : Natural := 0;
      West  : Natural := 0);

   procedure Turn
     (Self  : in out Ship;
      Left  : Natural := 0;
      Right : Natural := 0);

   procedure Forward
     (Self  : in out Ship;
      Value : Positive);

   function Manhattan_Distance (Self : Ship) return Natural;
end Part_2;
      
package body Part_2 is
   overriding procedure Move
     (Self  : in out Ship;
      North : Natural := 0;
      South : Natural := 0;
      East  : Natural := 0;
      West  : Natural := 0) is
   begin
      Self.Waypoint.North := Self.Waypoint.North + North - South;
      Self.Waypoint.East := Self.Waypoint.East + East - West;
   end Move;

   procedure Turn
     (Self  : in out Ship;
      Left  : Natural := 0;
      Right : Natural := 0) is
   begin
      case (Left - Right) mod 360 is
         when 0 =>
            null;
         when 90 =>
            Self.Waypoint :=
              (East  => -Self.Waypoint.North,
               North => Self.Waypoint.East);
         when 180 =>
            Self.Waypoint :=
              (East  => -Self.Waypoint.East,
               North => -Self.Waypoint.North);
         when 270 =>
            Self.Waypoint :=
              (East  => Self.Waypoint.North,
               North => -Self.Waypoint.East);
         when others =>
            raise Program_Error;
      end case;
   end Turn;

   procedure Forward
     (Self  : in out Ship;
      Value : Positive) is
   begin
      Self.Ship :=
        (East  => Self.Ship.East  + Self.Waypoint.East * Value,
         North => Self.Ship.North + Self.Waypoint.North * Value);
   end Forward;

   function Manhattan_Distance (Self : Ship) return Natural is
   begin
      return abs Self.Ship.East + abs Self.Ship.North;
   end Manhattan_Distance;
end Part_2;

Now we can declare two ships to drive them in parallel.

In [5]:
Ship_1 : Part_1.Ship;
Ship_2 : Part_2.Ship;
Input  : Ada.Text_IO.File_Type;

The drive reads the input and call corresponding methods for both ships.

In [6]:
Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/home/jovyan/md/12/input");

while not Ada.Text_IO.End_Of_File (Input) loop
   declare
      Command : Character;
      Argument : Positive;
   begin
      Ada.Text_IO.Get (Input, Command);
      Ada.Integer_Text_IO.Get (Input, Argument);
         
      case Command is
         when 'N' =>
            Ship_1.Move (North => Argument);
            Ship_2.Move (North => Argument);
         when 'S' =>
            Ship_1.Move (South => Argument);
            Ship_2.Move (South => Argument);
         when 'E' =>
            Ship_1.Move (East => Argument);
            Ship_2.Move (East => Argument);
         when 'W' =>
            Ship_1.Move (West => Argument);
            Ship_2.Move (West => Argument);
         when 'L' =>
            Ship_1.Turn (Left => Argument);
            Ship_2.Turn (Left => Argument);
         when 'R' =>
            Ship_1.Turn (Right => Argument);
            Ship_2.Turn (Right => Argument);
         when 'F' =>
            Ship_1.Forward (Argument);
            Ship_2.Forward (Argument);
         when others =>
            raise Program_Error;
      end case;
   end;
end loop;

Let's print results

In [7]:
Ada.Integer_Text_IO.Put (Ship_1.Manhattan_Distance);
Ada.Integer_Text_IO.Put (Ship_2.Manhattan_Distance);

        381      28591

----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020)